In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
! pip install git+https://github.com/LIAAD/yake
import yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-kvhhxsk5
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-kvhhxsk5


In [3]:
from collections import defaultdict
from math import log, sqrt
import re
import numpy as np
import sys
from copy import deepcopy
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import datetime

In [4]:
file1 = open("/content/drive/MyDrive/Tweelink_Dataset/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [5]:
def get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed):
  relevant_docs_list = []
  for doc in docs_preprocessed:
    if doc['Data']['Base Hashtag']==base_hashtag:
      current_date = datetime.datetime.strptime(base_date, format)
      prev_date = current_date - datetime.timedelta(days=1)
      next_date = current_date + datetime.timedelta(days=1)
      if(doc['Data']['Date'] in [str(prev_date.date()), str(current_date.date()), str(next_date.date())]):
        relevant_docs_list.append(doc['Name'])
  return relevant_docs_list

In [6]:
def precision_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr] in relevant_docs_list):
      num_of_relevant_results+=1
  return num_of_relevant_results/k

In [7]:
def mean_average_precision(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_precision=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    precision_at_k_val = precision_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    print('Hashtag: {}   Precision@{}: {}'.format(base_hashtag, k_val, precision_at_k_val))
    average_precision += precision_at_k_val
  return average_precision/ctr

In [8]:
def recall_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  current_num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr] in relevant_docs_list):
      current_num_of_relevant_results+=1
  if(len(relevant_docs_list)==0):
    return 0
  return current_num_of_relevant_results/len(relevant_docs_list)

In [9]:
def mean_average_recall(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_recall=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    recall_at_k_val = recall_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    print('Hashtag: {}   Recall@{}: {}'.format(base_hashtag, k_val, recall_at_k_val))
    average_recall += recall_at_k_val
  return average_recall/ctr

In [10]:
# ## Import the data ad create the inverted index

def import_dataset():
    """
    This function import all the articles in the TIME corpus,
    returning list of lists where each sub-list contains all the
    terms present in the document as a string.
    """
    # articles = []
    # with open('TIME.ALL', 'r') as f:
    #     tmp = []
    #     for row in f:
    #         if row.startswith("*TEXT"):
    #             if tmp != []:
    #                 articles.append(tmp)
    #             tmp = []
    #         else:
    #             row = re.sub(r'[^a-zA-Z\s]+', '', row)
    #             tmp += row.split()
    # return articles

    docs_preprocessed_metrics = []
    docs_preprocessed = []
    docs_preprocessed_with_names = []
    path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
    for filename in glob(os.path.join(path, '*')):
      with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
        filename = os.path.basename(f.name)
        data = json.load(f)
        d_date = data["Date"]
        if(d_date=="" or d_date=="Date"):
          continue
        format = '%Y-%m-%d'
    
        d_present_date = datetime.datetime.strptime(d_date, format)
    
        if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
          continue
      
        docs_preprocessed_metrics.append({'Name':filename, 'Data':data})
        docs_preprocessed.append(data['Body_processed'])
        docs_preprocessed_with_names.append(filename)

    return docs_preprocessed, docs_preprocessed_with_names, docs_preprocessed_metrics




# def remove_stop_words(corpus):
#    '''
#    This function removes from the corpus all the stop words present in the file TIME.STP
#    '''
#    stop_w = [line.rstrip('\n') for line in open('TIME.STP')]
#    stop_w=list(filter(None, stop_w))
#    for i in range(0,len(corpus)):
#        corpus[i] = [x for x in corpus[i] if x not in stop_w]
#    return corpus 


def make_inverted_index(corpus):
    """
    This function builds an inverted index as an hash table (dictionary)
    where the keys are the terms and the values are ordered lists of
    docIDs containing the term.
    """
    # corpus = remove_stop_words(corpus)
    index = defaultdict(set)
    for docid, article in enumerate(corpus):
        for term in article:
            index[term].add(docid)
    return index


# ### Union of two posting lists


def posting_lists_union(pl1, pl2):
        """
        Returns a new posting list resulting from the union of the
        two lists passed as arguments.
        """
        pl1 = sorted(list(pl1))
        pl2 = sorted(list(pl2))
        union = []
        i = 0
        j = 0
        while (i < len(pl1) and j < len(pl2)):
            if (pl1[i] == pl2[j]):
                union.append(pl1[i])
                i += 1
                j += 1
            elif (pl1[i] < pl2[j]):
                union.append(pl1[i])
                i += 1
            else:
                union.append(pl2[j])
                j += 1
        for k in range(i, len(pl1)):
            union.append(pl1[k])
        for k in range(j, len(pl2)):
            union.append(pl2[k])
        return union


# ## Precomputing weights


def DF(term, index):
    '''
    Function computing Document Frequency for a term.
    '''
    return len(index[term])


def IDF(term, index, corpus):
    '''
    Function computing Inverse Document Frequency for a term.
    '''
    return log(len(corpus)/DF(term, index))


def RSV_weights(corpus,index):
    '''
    This function precomputes the Retrieval Status Value weights
    for each term in the index
    '''
    N = len(corpus)
    w = {}
    for term in index.keys():
        p = DF(term, index)/(N+0.5)  
        w[term] = IDF(term, index, corpus) + log(p/(1-p))
    return w
    


# ## BIM Class


class BIM():
    '''
    Binary Independence Model class
    '''
    
    def __init__(self, corpus, articles_with_name):
        self.original_corpus = deepcopy(corpus)
        self.articles_with_name = articles_with_name
        self.articles = corpus
        self.index = make_inverted_index(self.articles)
        self.weights = RSV_weights(self.articles, self.index)
        self.ranked = []
        self.query_text = ''
        self.N_retrieved = 0
    
        
    def RSV_doc_query(self, doc_id, query):
        '''
        This function computes the Retrieval Status Value for a given couple document - query
        using the precomputed weights
        '''
        score = 0
        doc = self.articles[doc_id]
        for term in doc:
            if term in query:
                score += self.weights[term]     
        return score

    
        
    def ranking(self, query):
        '''
        Auxiliary function for the function answer_query. Computes the score only for documents
        that are in the posting list of al least one term in the query
        '''

        docs = []
        for term in self.index: 
            # print(term in query, query, term)
            if term in query:
                docs = posting_lists_union(docs, self.index[term])

        scores = []
        for doc in docs:
            scores.append((doc, self.RSV_doc_query(doc, query)))
        
        self.ranked = sorted(scores, key=lambda x: x[1], reverse = True)
        return self.ranked
    
    
    
    def recompute_weights(self, relevant_idx, query):
        '''
        Auxiliary function for relevance_feedback function and
        for the pseudo relevance feedback in answer_query function.
        Recomputes the weights, only for the terms in the query
        based on a set of relevant documents.
        '''
        
        relevant_docs = []
        for idx in relevant_idx:
            doc_id = self.ranked[idx-1][0]
            relevant_docs.append(self.articles[doc_id])
        
        N = len(self.articles)
        N_rel = len(relevant_idx)
        
        for term in query:
            if term in self.weights.keys():
                vri = 0
                for doc in relevant_docs:
                    if term in doc:
                        vri += 1
                p = (vri + 0.5) /( N_rel + 1)
                u = (DF(term, self.index) - vri + 0.5) / (N - N_rel +1)
                self.weights[term] = log((1-u)/u) + log(p/(1-p))

            
    
    def answer_query(self, query_text):
        '''
        Function to answer a free text query. Shows the first 30 words of the
        15 most relevant documents. 
        Also implements the pseudo relevance feedback with k = 5
        '''
        
        self.query_text = query_text
        query =  query_text.lower().split()
        ranking = self.ranking(query)

        ## pseudo relevance feedback 
        i = 0
        new_ranking=[]
        while i<10 and ranking != new_ranking:
            self.recompute_weights([1,2,3,4,5], query)
            new_ranking = self.ranking(query)
            i+=1


        ranking = new_ranking
        
        self.N_retrieved = 20
        
        ## print retrieved documents
        ret_list = []
        for i in range(0, self.N_retrieved):
            article = self.original_corpus[ranking[i][0]]
            if (len(article) > 30):
                article = article[0:30]
            text = " ".join(article)
            print(f"Article {self.articles_with_name[self.ranked[i][0]]}, score: {ranking[i][1]}")
            ret_list.append(self.articles_with_name[self.ranked[i][0]])
            # print(text, '\n')

        self.weights = RSV_weights(self.articles, self.index)
        return ret_list


            
    def relevance_feedback(self, *args):
        '''
        Function that implements relevance feedback for the last query formulated.
        The set of relevant documents is given by the user through a set of indexes
        '''
        if(self.query_text == ''):
            print('Cannot get feedback before a query is formulated.')
            return
        
        relevant_idx = list(args)
        
        if(isinstance(relevant_idx[0], list)):
            relevant_idx = relevant_idx[0]
        
        query = self.query_text.upper().split()
        self.recompute_weights(relevant_idx,query)
        
        self.answer_query(self.query_text)
    
    
    def read_document(self,rank_num):
        '''
        Function that allows the user to select a document among the ones returned 
        by answer_query and read the whole text
        '''
        if(self.query_text == ''):
            print('Cannot select a document before a query is formulated.')
            return
            
        article = self.original_corpus[self.ranked[rank_num - 1][0]]
        text = " ".join(article)
        print(f"Article {rank_num}, score: {self.ranked[rank_num][1]}")
        print(text, '\n')
        
        
    def show_more(self):
        '''
        Function that allows the user to see more 10 retrieved documents
        '''
        
        if(self.N_retrieved + 10 > len(self.ranked)):
            print('No more documents available')
            return 
        
        for i in range(self.N_retrieved, self.N_retrieved+10):
            article = self.original_corpus[self.ranked[i][0]]
            if (len(article) > 30):
                article = article[0:30]
            text = " ".join(article)
            print(f"Article {self.articles_with_name[self.ranked[i][0]]}, score: {self.ranked[i][1]}")
            print(text, '\n')
        
        self.N_retrieved += 10 
        



# Example of usage

# articles, articles_with_name = import_dataset()
# bim  = BIM(articles, articles_with_name)
# bim.answer_query('Italy and Great Britain fight the enemy')
# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

In [11]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: hijab
Enter time: 2022-02-19
Enter Location: india


In [12]:
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [13]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [14]:
tweet_keywords = []
kw_extractor = yake.KeywordExtractor(top=100, stopwords=None)
keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
for kw, v in keywords:
  # print("Keyphrase: ",kw, ": score", v)
  for key in kw.split():
    if(key not in tweet_keywords):
      tweet_keywords.append(key)

print(tweet_keywords)


['hijab', 'karnataka', 'india', 'muslim', 'islam', 'hijabisfundamentalright', 'hijabisourright', 'hijabcontroversy', 'started', 'abaya', 'hijabrow', 'http', 'persecution', 'wear', 'allah', 'woman', 'karnatakahijabcontroversy', 'wearing', 'school', 'world', 'modestfashion', 'quran', 'hijabisindividualright', 'judge', 'modest', 'deen', 'prophetmuhammad', 'dua', 'makkah', 'college', 'hijabban', 'religion', 'china', 'remove', 'islamophobia', 'girl', 'hindu', 'education', 'islamist', 'hijabisourpride', 'hijaboruniform', 'student', 'permission', 'beard', 'freedom', 'asian', 'allowed', 'public', 'saudi', 'hijabplot', 'hijabaurkitab', 'class', 'niqab', 'muslimmen', 'leader']


In [15]:
# Example of usage
articles, articles_with_name, docs_preprocessed_metrics = import_dataset()

In [16]:
#results with normal query
bim  = BIM(articles, articles_with_name)
relevant_articles_list_1 = bim.answer_query(" ".join(tweet_query))

# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

Article UkraineRussiaCrisis_212.json, score: 932.1615793235385
Article UkraineRussiaCrisis_264.json, score: 790.8933846296359
Article vaccine_299.json, score: 679.4266212521104
Article UkraineRussiaCrisis_213.json, score: 636.0073299018009
Article narendramodi_256.json, score: 588.133861214538
Article UkraineRussiaCrisis_261.json, score: 570.5954727445408
Article UkraineRussiaCrisis_220.json, score: 512.6806648481777
Article UkraineRussiaCrisis_215.json, score: 418.2201738243974
Article UkraineRussiaCrisis_270.json, score: 387.71954960765765
Article hijab_286.json, score: 374.4658480987304
Article UkraineRussiaCrisis_216.json, score: 324.95690417228866
Article hijab_285.json, score: 311.56960827872285
Article TaylorSwift_235.json, score: 310.28095151470364
Article vaccine_291.json, score: 308.59067375183935
Article UkraineRussiaCrisis_211.json, score: 308.56011700514296
Article hijab_284.json, score: 295.26180956009523
Article UkraineRussiaCrisis_266.json, score: 283.52551098503864
Art

In [17]:
mean_average_precision_hashtag = mean_average_precision(20, u_base_hashtag, u_time, relevant_articles_list_1, docs_preprocessed_metrics)
print(mean_average_precision_hashtag)

Hashtag: hijab   Precision@1: 0.0
Hashtag: hijab   Precision@2: 0.0
Hashtag: hijab   Precision@3: 0.0
Hashtag: hijab   Precision@4: 0.0
Hashtag: hijab   Precision@5: 0.0
Hashtag: hijab   Precision@6: 0.0
Hashtag: hijab   Precision@7: 0.0
Hashtag: hijab   Precision@8: 0.0
Hashtag: hijab   Precision@9: 0.0
Hashtag: hijab   Precision@10: 0.1
Hashtag: hijab   Precision@11: 0.09090909090909091
Hashtag: hijab   Precision@12: 0.16666666666666666
Hashtag: hijab   Precision@13: 0.15384615384615385
Hashtag: hijab   Precision@14: 0.14285714285714285
Hashtag: hijab   Precision@15: 0.13333333333333333
Hashtag: hijab   Precision@16: 0.1875
Hashtag: hijab   Precision@17: 0.17647058823529413
Hashtag: hijab   Precision@18: 0.16666666666666666
Hashtag: hijab   Precision@19: 0.15789473684210525
Hashtag: hijab   Precision@20: 0.15
0.08130721896782268


In [18]:
# results with top k keywords
bim  = BIM(articles, articles_with_name)
relevant_articles_list_2 = bim.answer_query(" ".join(tweet_keywords))

# bim.relevance_feedback(5,6,8)
# bim.show_more()
# bim.read_document(2)

Article hijab_287.json, score: 354.2969396047313
Article hijab_285.json, score: 303.2142699556743
Article hijab_286.json, score: 302.2618356805923
Article hijab_288.json, score: 293.202659097206
Article hijab_281.json, score: 252.95173934816063
Article hijab_284.json, score: 218.78264950179994
Article hijab_289.json, score: 212.4746252952575
Article hijab_282.json, score: 202.36480224012365
Article hijab_290.json, score: 161.09023758058015
Article hijab_283.json, score: 160.60603394208113
Article narendramodi_253.json, score: 18.670173258663034
Article UkraineRussiaCrisis_264.json, score: 18.61105938233506
Article UkraineRussiaCrisis_218.json, score: 16.915064008001238
Article IndiaFightsCorona_246.json, score: 14.857431629982603
Article UkraineRussiaCrisis_214.json, score: 14.322889165260447
Article narendramodi_255.json, score: 12.69521616601773
Article narendramodi_259.json, score: 9.55567938396042
Article IndiaFightsCorona_247.json, score: 9.5485927768403
Article IndiaFightsCorona_

In [19]:
mean_average_recall_hashtag = mean_average_recall(20, u_base_hashtag, u_time, relevant_articles_list_2, docs_preprocessed_metrics)
print(mean_average_recall_hashtag)

Hashtag: hijab   Recall@1: 0.05
Hashtag: hijab   Recall@2: 0.1
Hashtag: hijab   Recall@3: 0.15
Hashtag: hijab   Recall@4: 0.2
Hashtag: hijab   Recall@5: 0.25
Hashtag: hijab   Recall@6: 0.3
Hashtag: hijab   Recall@7: 0.35
Hashtag: hijab   Recall@8: 0.4
Hashtag: hijab   Recall@9: 0.45
Hashtag: hijab   Recall@10: 0.5
Hashtag: hijab   Recall@11: 0.5
Hashtag: hijab   Recall@12: 0.5
Hashtag: hijab   Recall@13: 0.5
Hashtag: hijab   Recall@14: 0.5
Hashtag: hijab   Recall@15: 0.5
Hashtag: hijab   Recall@16: 0.5
Hashtag: hijab   Recall@17: 0.5
Hashtag: hijab   Recall@18: 0.5
Hashtag: hijab   Recall@19: 0.5
Hashtag: hijab   Recall@20: 0.5
0.3875
